Preprocessing - **PCA** (component=4)  
Algorithm - **KNN Classification**  
Feature - **Magnitude** **(251)**

In [ ]:

import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

# Load dataset
measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]

# Train-test split
# First, split into train (60%) and temp (40%) sets
trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')
print(f"Evaluation Data: {evalCIR.shape}")

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)
n_comp = 6

trainData - (600, 15, 251, 2)
testData - (200, 15, 251, 2)
Evaluation Data: (200, 15, 251, 2)


In [ ]:
# -------------------------------------------------- Preprocessing ------------------------------------------
def apply_pca(data, n_components):
    
    reshaped_data = data.reshape(data.shape[0], -1)  
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

# -------- Train --------
# Feature Extraction
train_alice_cirs = trainCIR[:, alice_channel, :, :]
train_eve_cirs = trainCIR[:, eve_channel, :, :]

train_alice_mag = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_mag = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])

train_cirs = np.vstack((train_alice_mag, train_eve_mag))
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp) 
print(f'train_cirs_pca: {train_cirs_pca.shape}')

# -------- Test --------
# Feature Extraction
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_eve_cirs = testCIR[:, eve_channel, :, :]

test_alice_mag = np.abs(test_alice_cirs[..., 0] + 1j * test_alice_cirs[..., 1])
test_eve_mag = np.abs(test_eve_cirs[..., 0] + 1j * test_eve_cirs[..., 1])
test_cirs = np.vstack((test_alice_mag, test_eve_mag))

# Scale
test_cirs_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {test_cirs_scaled.shape}')
# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)
print(f'test_cirs_pca: {test_cirs_pca.shape}')

# ----------- Labels -----------
# train
train_alice_labels = np.zeros(train_alice_cirs.shape[0])
train_eve_labels = np.ones(train_eve_cirs.shape[0])
train_labels = np.hstack((train_alice_labels, train_eve_labels))
# test
test_alice_label = np.zeros(test_alice_cirs.shape[0])
test_eve_labels = np.ones(test_eve_cirs.shape[0])
test_labels = np.hstack((test_alice_label, test_eve_labels))


reshaped_data: (1200, 251)
data_scaled: (1200, 251)
train_cirs_pca: (1200, 6)
test_cirs: (400, 251)
test_cirs_scaled: (400, 251)
test_cirs_pca: (400, 6)


In [ ]:

# ----------------- Classification -----------------

# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_cirs_pca, train_labels)

# Predict on test data
predictions = classifier.predict(test_cirs_pca)

In [13]:
# ----------------- Testing Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 76.50%
tp: 152
tn: 154
fp: 46
fn: 48
MDR: 0.23
FAR: 0.24
AR: 0.765


In [14]:
# -------------------------------------------------------- Evaluation --------------------------------------------------------
# Feature Extraction
eval_alice_cirs = evalCIR[:, alice_channel, :, :]
eval_eve_cirs = evalCIR[:, eve_channel, :, :]

eval_alice_mag = np.abs(eval_alice_cirs[..., 0] + 1j * eval_alice_cirs[..., 1])
eval_eve_mag = np.abs(eval_eve_cirs[..., 0] + 1j * eval_eve_cirs[..., 1])
eval_cirs = np.vstack((eval_alice_mag, eval_eve_mag))

# Scale evaluation data
eval_cirs_scaled = scaler.transform(eval_cirs)

# Apply PCA to evaluation data
eval_cirs_pca = pca.transform(eval_cirs_scaled)

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_cirs.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_cirs.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Predict on evaluation data
eval_predictions = classifier.predict(eval_cirs_pca)

# Calculate accuracy
eval_accuracy  = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")



Classification Accuracy: 77.25%
tp: 155
tn: 154
fp: 46
fn: 45
MDR: 0.23
FAR: 0.225
AR: 0.7725


In [15]:
# import matplotlib.pyplot as plt
# accu = [70.75, 71.00, 73.25, 74.00, 75.50]
# comp = [2, 3, 4, 5, 6]

# plt.figure(figsize=(10, 6))
# plt.plot(accu, comp, marker='o', linestyle='-', color='b')
# plt.xlabel('Classification Accuracy (%)')
# plt.ylabel('Number of PCA Components')
# plt.title('Classification Accuracy vs. PCA Components')
# plt.yticks(comp)  # Set y-axis ticks to match the components values
# plt.grid(True)
# plt.show()

# method = ["Sparse Classifier", "OneClassSVM", "SVM", "Sparse data + SVM"]
# acc = [53.50, 61.25, 75.75, 84.50]

# plt.figure(figsize=(10, 6))
# plt.plot(method, acc, marker='o', linestyle='-', color='b')
# plt.xlabel('Method')
# plt.ylabel('Accuracy (%)')
# plt.title('Classification Accuracy vs. Methods')
# # plt.yticks(acc)  # Set y-axis ticks to match the components values
# plt.grid(True)
# plt.show()